In [ ]:
#@title 単体ファイル解析
from moviepy.editor import VideoFileClip
import numpy as np
import cv2
import librosa
import librosa.display
import matplotlib.pyplot as plt
from google.colab import files
import tempfile
import os

# 動画アップロード
uploaded = files.upload()
video_path = next(iter(uploaded))
print(f"Uploaded: {video_path}")

# 動画→音声を一時ファイルに抽出
clip = VideoFileClip(video_path)
audio = clip.audio
fps = clip.fps
duration = clip.duration
total_frames = int(duration * fps)

temp_wav_path = tempfile.mktemp(suffix=".wav")
audio.write_audiofile(temp_wav_path, fps=44100, verbose=False, logger=None)

# librosaで音声読み込み
y, sr = librosa.load(temp_wav_path, sr=44100)

# パラメータ設定
hop_length = 512
n_fft = 2048
stft = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
spectrogram = np.abs(stft)

# 周波数と時間の配列取得
frequencies = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
times = librosa.frames_to_time(np.arange(spectrogram.shape[1]), sr=sr, hop_length=hop_length)

# 351Hzに最も近いインデックスを探す
target_freq = 351
freq_index = np.argmin(np.abs(frequencies - target_freq))

# 該当周波数の強さを時間ごとに抽出
target_magnitudes = spectrogram[freq_index, :]
max_index = np.argmax(target_magnitudes)
max_time = times[max_index]
max_frame = int(max_time * fps)

print(f"📍 Max 351Hz magnitude at time: {max_time:.2f}s (frame {max_frame})")

# OpenCVで該当フレーム抽出
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, max_frame)
ret, frame = cap.read()

if ret:
    output_path = "frame_max_351Hz.jpg"
    cv2.imwrite(output_path, frame)
    print(f"✅ Frame with max 351Hz saved as: {output_path}")
    files.download(output_path)
else:
    print("❌ フレームの取得に失敗しました")

cap.release()

# スペクトログラムのdB変換（全体）
D_full = librosa.amplitude_to_db(np.abs(stft), ref=np.max)

plt.figure(figsize=(14, 6))
librosa.display.specshow(D_full, sr=sr, hop_length=hop_length, x_axis='time', y_axis='log', cmap='magma')
plt.colorbar(format='%+2.0f dB')
plt.title("Full Spectrogram with 351Hz Peak Marker")
plt.ylim(300, 400)  # 351Hz付近だけを表示（必要に応じて広げてもOK）
plt.axvline(max_time, color='cyan', linestyle='--', label='max 351Hz')  # ピーク位置に線を引く
plt.legend()
plt.show()

  if event.key is 'enter':



TypeError: 'NoneType' object is not subscriptable

In [ ]:
#@title 複数ファイル解析
from moviepy.editor import VideoFileClip
import numpy as np
import cv2
import librosa
import librosa.display
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive
import tempfile
import os

drive.mount('/content/drive')
# Google Driveに保存
drive_output_dir_fr = '/content/drive/MyDrive/実験2/最大フレーム写真'
drive_output_dir_sp = '/content/drive/MyDrive/実験2/スペクトログラム'

# 動画複数アップロード
uploaded = files.upload()

for video_path in uploaded:
    print(f"\n Processing: {video_path}")

    # 動画読み込み
    clip = VideoFileClip(video_path)
    audio = clip.audio
    fps = clip.fps
    duration = clip.duration
    total_frames = int(duration * fps)

    # 一時wavファイルに音声保存
    temp_wav_path = tempfile.mktemp(suffix=".wav")
    audio.write_audiofile(temp_wav_path, fps=44100, verbose=False, logger=None)

    # librosaで音声読み込み
    y, sr = librosa.load(temp_wav_path, sr=44100)

    # STFT計算
    hop_length = 512
    n_fft = 2048
    stft = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    spectrogram = np.abs(stft)

    frequencies = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    times = librosa.frames_to_time(np.arange(spectrogram.shape[1]), sr=sr, hop_length=hop_length)

    # 351Hzのインデックス
    target_freq = 351
    freq_index = np.argmin(np.abs(frequencies - target_freq))

    # 該当周波数の強さ
    target_magnitudes = spectrogram[freq_index, :]
    max_index = np.argmax(target_magnitudes)
    max_time = times[max_index]
    max_frame = int(max_time * fps)

    print(f"351HzMaxVolume 時刻: {max_time:.2f}s (フレーム番号 {max_frame})")

    # フレーム抽出
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, max_frame)
    ret, frame = cap.read()

    base_name = os.path.splitext(os.path.basename(video_path))[0]

    if ret:
        frame_output_path = os.path.join(drive_output_dir_fr, f"{base_name}.max_volume_frame.jpg")
        cv2.imwrite(frame_output_path, frame)
        print(f"フレームを保存しました: {frame_output_path}")
    else:
        print("フレームの取得に失敗しました")

    cap.release()

    # スペクトログラム作成
    D_full = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    plt.figure(figsize=(14, 6))
    librosa.display.specshow(D_full, sr=sr, hop_length=hop_length,
                              x_axis='time', y_axis='log', cmap='magma')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Spectrogram")
    plt.ylim(300, 400)
    plt.axvline(max_time, color='cyan', linestyle='--', label='max 351Hz')
    plt.legend()

    # グラフ保存
    spectrogram_path = os.path.join(drive_output_dir_sp, f"{base_name}.spectrogram.png")
    plt.savefig(spectrogram_path)
    plt.close()
    print(f"スペクトログラムを保存しました: {spectrogram_path}")

print("\n✅ すべての処理が完了しました")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving 5月27日実験6の1.mp4 to 5月27日実験6の1 (1).mp4
Saving 5月27日実験6の2.mp4 to 5月27日実験6の2 (1).mp4
Saving 5月27日実験7の1.mp4 to 5月27日実験7の1.mp4
Saving 5月27日実験7の2.mp4 to 5月27日実験7の2.mp4
Saving 5月28日実験1の1.mp4 to 5月28日実験1の1.mp4

 Processing: 5月27日実験6の1 (1).mp4
351HzMaxVolume 時刻: 2.69s (フレーム番号 80)
フレームを保存しました: /content/drive/MyDrive/実験2/最大フレーム写真/5月27日実験6の1 (1).max_volume_frame.jpg


  plt.savefig(spectrogram_path)



スペクトログラムを保存しました: /content/drive/MyDrive/実験2/スペクトログラム/5月27日実験6の1 (1).spectrogram.png

 Processing: 5月27日実験6の2 (1).mp4
351HzMaxVolume 時刻: 2.10s (フレーム番号 63)
フレームを保存しました: /content/drive/MyDrive/実験2/最大フレーム写真/5月27日実験6の2 (1).max_volume_frame.jpg
スペクトログラムを保存しました: /content/drive/MyDrive/実験2/スペクトログラム/5月27日実験6の2 (1).spectrogram.png

 Processing: 5月27日実験7の1.mp4
351HzMaxVolume 時刻: 1.99s (フレーム番号 59)
フレームを保存しました: /content/drive/MyDrive/実験2/最大フレーム写真/5月27日実験7の1.max_volume_frame.jpg
スペクトログラムを保存しました: /content/drive/MyDrive/実験2/スペクトログラム/5月27日実験7の1.spectrogram.png

 Processing: 5月27日実験7の2.mp4
351HzMaxVolume 時刻: 1.53s (フレーム番号 45)
フレームを保存しました: /content/drive/MyDrive/実験2/最大フレーム写真/5月27日実験7の2.max_volume_frame.jpg
スペクトログラムを保存しました: /content/drive/MyDrive/実験2/スペクトログラム/5月27日実験7の2.spectrogram.png

 Processing: 5月28日実験1の1.mp4
351HzMaxVolume 時刻: 4.11s (フレーム番号 122)
フレームを保存しました: /content/drive/MyDrive/実験2/最大フレーム写真/5月28日実験1の1.max_volume_frame.jpg
スペクトログラムを保存しました: /content/drive/MyDrive/実験2/スペクトログラム/5月28日実験1の1.spectrogram

In [ ]:
#@title アップロードしたファイル削除
import os

for filename in os.listdir():
    if os.path.isfile(filename):
        os.remove(filename)
        print(f"削除したファイル: {filename}")